<a href="https://colab.research.google.com/github/Michael-Dyq/timeline_generation/blob/main/MATRES_test_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
with open('/content/drive/MyDrive/Colab Notebooks/RA/Time/t5_timeline/MATRES/platinum.txt', 'r') as rf:
  t3_label = []
  lines = rf.readlines()
  for line in lines:
    t3_label.append(line.split())
  print(len(t3_label))

837


In [ ]:
t3_file_names = [x[0] for x in t3_label]
t3_file_names = set(t3_file_names)
len(t3_file_names) #len = 20
t3_file_names

In [7]:
t3_dic = {}
c = 0
for key in t3_file_names:
  cur_list = []
  for i in t3_label:
    if i[0] == key:
      cur_list.append(i)
      c += 1
  t3_dic[key] = cur_list
print(c)

837


In [ ]:
t3_dic['AP_20130322']

In [8]:
import re
import os
import random

In [11]:
doc_name = t3_label[0][0] + '.tml'
folder_path = '/content/drive/MyDrive/Colab Notebooks/RA/Time/t5_timeline/MATRES/te3-platinum/'

with open(os.path.join(folder_path, doc_name), 'r') as f:
    data = f.read()

In [32]:
matres_test_dataset = []
folder_path = '/content/drive/MyDrive/Colab Notebooks/RA/Time/t5_timeline/MATRES/te3-platinum/'
for doc_name in t3_dic.keys():
    doc_name_suffix = doc_name +'.tml'
    with open(os.path.join(folder_path, doc_name_suffix), 'r') as f:
        data = f.read()
    # get the text body
    start = data.find('<TEXT>') + 6
    end = data.find('</TEXT>') 
    text_tml = data[start:end].replace('\n', ' ')
    # iterate each event pair in the current document
    for ins in t3_dic[doc_name]:
        if ins[5] == 'AFTER' or ins[5] == 'BEFORE':
            rand_maker = ['<extra_' + str(i) + '>' for i in range(31)]
            random.shuffle(rand_maker)
            event1 = ins[1]
            event2 = ins[2]
            event1_id = ins[3]
            event2_id = ins[4]
            relation = ins[5]
            event1_start = text_tml.find('eid="e'+str(event1_id)+'"') + 8 + len(str(event1_id)) 
            event1_end = event1_start+len(event1)
            # insert <extra_id> behind the event 1 
            if text_tml[event1_start:event1_end] == event1:
                #print(text_tml[:event1_end] + " " + rand_maker[0] + text_tml[event1_end:])
                text_tml = text_tml[:event1_end] + " " + rand_maker[0] + text_tml[event1_end:]
            else: 
                print('event_1 does not match!')
            # insert <extra_id> behind the event 22
            event2_start = text_tml.find('eid="e'+str(event2_id)+'"') + 8 + len(str(event2_id)) 
            event2_end = event2_start+len(event2)
            if text_tml[event2_start:event2_end] == event2:
                #print(text_tml[:event2_end] + " " + rand_maker[1] + text_tml[event2_end:])
                text_tml = text_tml[:event2_end] + " " + rand_maker[1] + text_tml[event2_end:]
            else:
                print('event_2 does not match!')
                print(ins)
                continue
            # remove <EVENT> and <TIMEX3> in the text
            re_event=re.compile('</?EVENT.*?>')
            re_timex3=re.compile('</?TIMEX3.*?>')
            clean_text=re_event.sub('',text_tml)
            clean_text=re_timex3.sub('',clean_text)

            # add prefixt 'timeline generate'
            processed_text= 'timeline generate:' + clean_text

            # generate label
            if relation == "BEFORE":
                label = rand_maker[0] + ', ' + rand_maker[1]
            else:
                label = rand_maker[1] + ', ' + rand_maker[0]

            matres_test_dataset.append((processed_text, label))
            
        else:
            continue

len(matres_test_dataset)     

event_2 does not match!
['CNN_20130321_821', 'unleashed', 'intended', '8', '1000', 'BEFORE']


692

In [33]:
matres_test_dataset[0][0]

'timeline generate:  Israeli Prime Minister Benjamin Netanyahu apologized <extra_0> on Friday to Turkish Prime Minister Recep Tayyip Erdogan for a raid on a Turkish flotilla ship, a move that will help restore strained ties between the countries.  The call, which happened as President Barack Obama wrapped <extra_27> up his first presidential visit to Israel, was an unexpected outcome from a Mideast trip that seemed to yield few concrete steps.  Mr. Netanyahu\'s office confirmed that the Israeli leader, in a conversation with Mr. Erdogan, "agreed to restore normalization between Israel and Turkey, including the dispatch of ambassadors and the cancellation of legal steps against [Israeli Defense Forces] soldiers." Mr. Erdogan accepted the Israeli apology, the prime minister\'s office said.  Mr. Erdogan has long sought an apology for the raid in May 2010 on the Mavi Marmara, which was part of a flotilla that sought to break Israel\'s blockade of Gaza. An Israeli raid on the ship left nine

In [35]:
matres_test_dataset[0][1]

'<extra_0>, <extra_27>'

# Draft

In [12]:
start = data.find('<TEXT>') + 6
end = data.find('</TEXT>') 
#print(data[start:end])
text_tml = data[start:end].replace('\n', '')

# for an event pair 
rand_maker = ['<extra_' + str(i) + '>' for i in range(31)]
random.shuffle(rand_maker)
event1 = t3_label[0][1]
event2 = t3_label[0][2]
event1_id = t3_label[0][3]
event2_id = t3_label[0][4]
relation = t3_label[0][5]
event1_start = text_tml.find('eid="e'+str(event1_id)+'"') + 8 + len(str(event1_id)) 
event1_end = event1_start+len(event1)

#if before or after
if text_tml[event1_start:event1_end] == event1:
  print('event_1 match')
  print(text_tml[:event1_end] + " " + rand_maker[0] + text_tml[event1_end:])
  text_tml = text_tml[:event1_end] + " " + rand_maker[0] + text_tml[event1_end:]

event2_start = text_tml.find('eid="e'+str(event2_id)+'"') + 8 + len(str(event2_id)) 
event2_end = event2_start+len(event2)

if text_tml[event2_start:event2_end] == event2:
  print('event_2 match')
  print(text_tml[:event2_end] + " " + rand_maker[1] + text_tml[event2_end:])
  text_tml = text_tml[:event2_end] + " " + rand_maker[1] + text_tml[event2_end:]

# 去掉多余的<>
# + timeline generate
# 生成 label

event_1 match
Israeli Prime Minister Benjamin Netanyahu <EVENT class="OCCURRENCE" eid="e1">apologized <extra_27></EVENT> on <TIMEX3 tid="t1" type="DATE" value="2013-03-22">Friday</TIMEX3> to Turkish Prime Minister Recep Tayyip Erdogan for a <EVENT class="OCCURRENCE" eid="e2">raid</EVENT> on a Turkish flotilla ship, a <EVENT class="OCCURRENCE" eid="e56">move</EVENT> that will <EVENT class="I_ACTION" eid="e3">help</EVENT> <EVENT class="OCCURRENCE" eid="e4">restore</EVENT> strained ties between the countries.The <EVENT class="OCCURRENCE" eid="e57">call</EVENT>, which <EVENT class="OCCURRENCE" eid="e5">happened</EVENT> as President Barack Obama <EVENT class="OCCURRENCE" eid="e6">wrapped</EVENT> up his first presidential <EVENT class="OCCURRENCE" eid="e7">visit</EVENT> to Israel, was an unexpected <EVENT class="OCCURRENCE" eid="e8">outcome</EVENT> from a Mideast <EVENT class="OCCURRENCE" eid="e9">trip</EVENT> that <EVENT class="I_STATE" eid="e10">seemed</EVENT> to <EVENT class="OCCURRENCE" 

In [15]:
text_tml
re_event=re.compile('</?EVENT.*?>')
re_timex3=re.compile('</?TIMEX3.*?>')
clean_text=re_event.sub('',text_tml)
clean_text=re_timex3.sub('',clean_text)
clean_text

'Israeli Prime Minister Benjamin Netanyahu apologized <extra_27> on Friday to Turkish Prime Minister Recep Tayyip Erdogan for a raid on a Turkish flotilla ship, a move that will help restore strained ties between the countries.The call, which happened <extra_6> as President Barack Obama wrapped up his first presidential visit to Israel, was an unexpected outcome from a Mideast trip that seemed to yield few concrete steps.Mr. Netanyahu\'s office confirmed that the Israeli leader, in a conversation with Mr. Erdogan, "agreed to restore normalization between Israel and Turkey, including the dispatch of ambassadors and the cancellation of legal steps against [Israeli Defense Forces] soldiers." Mr. Erdogan accepted the Israeli apology, the prime minister\'s office said.Mr. Erdogan has long sought an apology for the raid in May 2010 on the Mavi Marmara, which was part of a flotilla that sought to break Israel\'s blockade of Gaza. An Israeli raid on the ship left nine passengers dead, all of t

In [ ]:
#rand_maker = ['<extra_' + str(i) + '>' for i in range(31)]
#random.shuffle(rand_maker)
rand_maker

In [10]:
start = data.find('<TEXT>') + 6
end = data.find('</TEXT>') 
print(data[start:end])
text_tml = data[start:end]



The flu <EVENT class="OCCURRENCE" eid="e1000028">season</EVENT> is winding down, and it has <EVENT class="OCCURRENCE" eid="e2">killed</EVENT> 105 children so far - about the average toll.

The season <EVENT class="ASPECTUAL" eid="e3">started</EVENT> about <TIMEX3 tid="t1" type="DURATION" value="P1M">a month</TIMEX3> earlier than usual, <EVENT class="ASPECTUAL" eid="e4">sparking</EVENT> concerns it might <EVENT class="OCCURRENCE" eid="e5">turn</EVENT> into the worst in <TIMEX3 type="DURATION" value="P10Y" tid="t1000">a decade</TIMEX3>. It <EVENT class="ASPECTUAL" eid="e6">ended</EVENT> up being very hard on the elderly, but was moderately severe overall, <EVENT class="REPORTING" eid="e7">according</EVENT> to the Centers for Disease Control and Prevention.

Six of the pediatric <EVENT class="OCCURRENCE" eid="e1000030">deaths</EVENT> were <EVENT class="REPORTING" eid="e8">reported</EVENT> in <TIMEX3 tid="t2" type="DATE" value="2013-W11">the last week</TIMEX3>, and it's possible there wi

In [ ]:
text_tml

'Israeli Prime Minister Benjamin Netanyahu <EVENT class="OCCURRENCE" eid="e1">apologized <extra_28></EVENT> on <TIMEX3 tid="t1" type="DATE" value="2013-03-22">Friday</TIMEX3> to Turkish Prime Minister Recep Tayyip Erdogan for a <EVENT class="OCCURRENCE" eid="e2">raid</EVENT> on a Turkish flotilla ship, a <EVENT class="OCCURRENCE" eid="e56">move</EVENT> that will <EVENT class="I_ACTION" eid="e3">help</EVENT> <EVENT class="OCCURRENCE" eid="e4">restore</EVENT> strained ties between the countries.The <EVENT class="OCCURRENCE" eid="e57">call</EVENT>, which <EVENT class="OCCURRENCE" eid="e5">happened <extra_23></EVENT> as President Barack Obama <EVENT class="OCCURRENCE" eid="e6">wrapped</EVENT> up his first presidential <EVENT class="OCCURRENCE" eid="e7">visit</EVENT> to Israel, was an unexpected <EVENT class="OCCURRENCE" eid="e8">outcome</EVENT> from a Mideast <EVENT class="OCCURRENCE" eid="e9">trip</EVENT> that <EVENT class="I_STATE" eid="e10">seemed</EVENT> to <EVENT class="OCCURRENCE" ei

In [ ]:
event2_id

'5'

In [ ]:
match = re.match('Prime', text_tml)
if match:
  print('yes')
  print(match.group())
